# Jump process using AlgebraicPetri.jl
Micah Halter (@mehalter), 2020-07-13

## Introduction

This implementation defines the model as the composition of two interactions defined at domain-level semantics, transmission and recovery, simulated using `DiffEqJump.jl`.

## Libraries

In [ ]:
using AlgebraicPetri.Epidemiology
using Petri
using Catlab.Theories
using Catlab.CategoricalAlgebra.ShapeDiagrams
using Catlab.Graphics
using DiffEqJump
using Random
using DataFrames
using StatsPlots
using BenchmarkTools
import Base: ≤

# helper function to visualize categorical representation
display_wd(ex) = to_graphviz(ex, orientation=LeftToRight, labels=true);

## Transitions

Using the categorical framework provided by the AlgebraicJulia environment, we
can think of building models as the combination of known building block open
models.  For example we have $transmission: S \otimes I \rightarrow I$ and
$recovery: I \rightarrow R$ interactions defined in the Epidemiology module of
AlgebraicPetri which can be visualized as the following Petri nets.

Transmission (where $S_1 = S$ and $S_2 = I$):

In [ ]:
Graph(decoration(F_epi(transmission)))

Recovery (where $S_1 = I$ and $S_2 = R$):

In [ ]:
Graph(decoration(F_epi(recovery)))

In this approach we can think of an sir model as the composition of transmission
and recovery. This allows us to define the relationship that the infected
population coming out of the transmission interaction is the same as population
of infected in the recovery interaction.

In [ ]:
sir_wiring_diagram = transmission ⋅ recovery
display_wd(sir_wiring_diagram)

using the function `F_epi` provided by `AlgebraicPetri.Epidemiology`, we can
convert this categorical definition of SIR to the Petri net representation and
visualize the newly created model (where $S_1 = S$, $S_2 = I$, and $S_3 = R$).

In [ ]:
sir_model = decoration(F_epi(sir_wiring_diagram));
Graph(sir_model)

## Time domain

In [ ]:
tmax = 40.0
tspan = (0.0,tmax);

For plotting, we can also define a separate time series.

In [ ]:
δt = 0.1
t = 0:δt:tmax;

## Initial conditions

In [ ]:
u0 = [990,10,0]; # S,I,R

## Parameter values

In [ ]:
p = [0.05*10.0/sum(u0),0.25]; # β*c/N,γ

## Random number seed

We set a random number seed for reproducibility.

In [ ]:
Random.seed!(1234);

## Running the model

Running this model involves:

- Setting up the problem as a `JumpProblem`;
- Running the model, specifying `SSAStepper`

In [ ]:
prob_jump = JumpProblem(sir_model, u0, tspan, p)

In [ ]:
sol_jump = solve(prob_jump,SSAStepper());

## Post-processing

In order to get output comparable across implementations, we output the model at a fixed set of times.

In [ ]:
out_jump = sol_jump(t);

We can convert to a dataframe for convenience.

In [ ]:
df_jump = DataFrame(out_jump')
df_jump[!,:t] = out_jump.t;

## Plotting

We can now plot the results.

In [ ]:
@df df_jump plot(:t,
    [:x1 :x2 :x3],
    label=["S" "I" "R"],
    xlabel="Time",
    ylabel="Number")

## Benchmarking

In [ ]:
@benchmark solve(prob_jump,SSAStepper())